# Gen V2 — Shift Scheduler Walkthrough

Блокнот демонстрирует, как модуль `scheduler.py` строит кандидатов, назначает работников и формирует отчёт. Запускайте его из корня репозитория, чтобы относительные пути к CSV разрешались автоматически.

## 1. Импорты

In [40]:
import pandas as pd

from scheduler import DataPipeline, AssignmentEngine, SchedulerReport

## 2. Загружаем справочники и план

Все входные CSV лежат в корне и используются как GUI, так и пайплайном.

In [ ]:
workers = pd.read_csv('data/workers.csv')
equipment = pd.read_csv('data/equipment.csv')
schedule = pd.read_csv('data/assignment_history.csv')
requirements = pd.read_csv('data/position_requirements.csv')
plan = pd.read_csv('data/plan.csv')

print(f'workers: {workers.shape}')
print(f'equipment: {equipment.shape}')
print(f'history assignments: {schedule.shape}')
print(f'requirements: {requirements.shape}')
print(f'plan: {plan.shape}')
workers.head()

workers: (60, 5)
equipment: (26, 3)
history assignments: (180, 6)
requirements: (9, 4)
plan: (47, 5)


,worker_id,name,flat_printing,letterpress_printing,inkjet_printing
0,W001,Работник П 01,7,6,5
1,W002,Работник П 02,6,5,0
2,W003,Работник П 03,5,4,0
3,W004,Работник П 04,4,0,5
4,W005,Работник П 05,7,6,0


## 3. Выбираем целевую неделю

GUI рассчитывает номер недели по выбранному понедельнику. Здесь берём максимальное значение из `plan.csv`, но можно задать конкретное вручную.

In [42]:
target_week = 46 # int(plan['week'].max())
print(f'Целевая неделя: {target_week}')

Целевая неделя: 46


## 4. DataPipeline — кандидаты и слоты

`DataPipeline` подготавливает список работников с учётом ротации смен и создаёт пустые слоты для каждой смены.

In [43]:
pipeline = DataPipeline(
    workers,
    equipment,
    schedule[['worker_id', 'week', 'shift']],
    requirements,
    plan,
)
pipeline.run(target_week)

print('Кандидаты на неделю:')
pipeline.shift_candidates.head()

Кандидаты на неделю:


,worker_id,week,prev_shift,shift,name,flat_printing,letterpress_printing,inkjet_printing,primary_profession,all_professions
0,W021,46,day,night,Работник П 21,7,6,0,flat_printing,"[flat_printing, letterpress_printing]"
1,W022,46,day,night,Работник П 22,6,5,0,flat_printing,"[flat_printing, letterpress_printing]"
2,W023,46,day,night,Работник П 23,5,4,0,flat_printing,"[flat_printing, letterpress_printing]"
3,W037,46,day,night,Работник О 37,5,5,5,flat_printing,"[flat_printing, letterpress_printing, inkjet_p..."
4,W025,46,day,night,Работник П 25,7,6,0,flat_printing,"[flat_printing, letterpress_printing]"


In [44]:
print('Слоты дневной смены:')
pipeline.shift_equipment_day.head()

Слоты дневной смены:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,46,day,PM-01,flat_printing,1,7,Печатник,None
1,46,day,PM-01,flat_printing,2,6,Печатник,None
2,46,day,PM-01,flat_printing,3,5,Печатник,None
3,46,day,PM-01,flat_printing,4,4,Печатник,None
4,46,day,PM-02,flat_printing,1,7,Печатник,None


## 5. AssignmentEngine — назначение работников

Движок выполняет серию туров по каждой смене и учитывает глобальный пул уже занятых работников.

In [45]:
engine = AssignmentEngine(
    pipeline.shift_candidates,
    pipeline.shift_equipment_day.copy(),
    pipeline.shift_equipment_evening.copy(),
    pipeline.shift_equipment_night.copy(),
)
engine.run()

print(f'Назначено работников: {len(engine.global_assigned)}')

Назначено работников: 60


### 5.1 Визуализация туров назначения

Ниже повторяем логику `AssignmentEngine`: видим, какие раунды (mode + shift) закрывают слоты дневной/вечерней/ночной смен. История показывает, сколько позиций заполнили после каждого тура и сколько осталось свободных.

In [46]:
default_tourse = [
    ("ferst", "day"),
    ("second", "day"),
    ("ferst", "night"),
    ("second", "night"),
    ("ferst", "evening"),
    ("second", "evening"),
]

default_tourse_day = default_tourse.copy()
default_tourse_evening = default_tourse[4:] + default_tourse[:4]
default_tourse_night = default_tourse[2:] + default_tourse[:2]

rounds_by_shift = {
    "day": default_tourse_day,
    "evening": default_tourse_evening,
    "night": default_tourse_night,
}


def visualize_shift_rounds(target_shift: str):
    """Возвращает DataFrame с историей туров и снапшот таблицы смены."""
    demo_engine = AssignmentEngine(
        pipeline.shift_candidates,
        pipeline.shift_equipment_day.copy(),
        pipeline.shift_equipment_evening.copy(),
        pipeline.shift_equipment_night.copy(),
    )

    assigned = getattr(demo_engine, f"assigned_{target_shift}").copy()
    shift_table = getattr(demo_engine, f"shift_equipment_{target_shift}").copy()
    rounds = rounds_by_shift[target_shift]

    history = []
    free = shift_table
    updated = shift_table.copy()

    for idx, (mode, candidate_shift) in enumerate(rounds, start=1):
        free, patch, assigned = demo_engine._fill_positions(
            free,
            assigned,
            mode=mode,
            shift_name=candidate_shift,
        )
        updated = updated.combine_first(patch)
        history.append(
            {
                "round": idx,
                "mode": mode,
                "candidates_from": candidate_shift,
                "filled_total": int(updated["worker_id"].notna().sum()),
                "remaining_slots": int(free.shape[0]),
            }
        )
        if free.empty:
            break

    return pd.DataFrame(history), updated


In [47]:
history_day, day_snapshot = visualize_shift_rounds("day")
history_day

,round,mode,candidates_from,filled_total,remaining_slots
0,1,ferst,day,16,4
1,2,second,day,20,0


In [48]:
history_evening, _ = visualize_shift_rounds("evening")
history_evening

,round,mode,candidates_from,filled_total,remaining_slots
0,1,ferst,evening,16,4
1,2,second,evening,20,0


In [49]:
history_night, _ = visualize_shift_rounds("night")
history_night

,round,mode,candidates_from,filled_total,remaining_slots
0,1,ferst,night,16,4
1,2,second,night,20,0


In [50]:
print('Не назначены (фрагмент):')
cols = ['worker_id', 'name', 'primary_profession', 'all_professions']
engine.no_position[cols].head()

Не назначены (фрагмент):


,worker_id,name,primary_profession,all_professions


## 6. SchedulerReport — финальные таблицы и текстовый отчёт

In [51]:
scheduler_report = SchedulerReport(
    engine.shift_equipment_night,
    engine.shift_equipment_day,
    engine.shift_equipment_evening,
    workers,
    pipeline.shift_candidates,
    engine.global_assigned,
    pipeline.plan_long,
)

scheduler_report.get_final_assignments()
scheduler_report.get_brigade_summary()
scheduler_report.generate_text_summary(target_week)

scheduler_report.final_assignments_df.head()

,week,shift,machine_id,position,worker_id,name
0,46,day,PM-01,1,W001,Работник П 01
1,46,day,PM-01,2,W002,Работник П 02
2,46,day,PM-01,3,W003,Работник П 03
3,46,day,PM-01,4,W017,Работник О 17
4,46,day,PM-02,1,W005,Работник П 05


In [52]:
print('Сводка по бригадам:')
scheduler_report.report.head()

Сводка по бригадам:


,week,shift,machine_id,machine_type,required,assigned
0,46,day,PM-01,flat_printing,4,4
1,46,day,PM-02,flat_printing,4,4
2,46,day,PM-03,letterpress_printing,4,4
3,46,day,PM-04,letterpress_printing,4,4
4,46,day,SM-01,inkjet_printing,1,1


In [53]:
print('\n'.join(scheduler_report.summary_lines))

--- РАБОТНИКИ ---
Целевая неделя: 46
Всего доступно: 60
Назначено на смены: 60
Остались без смены: 0

--- ПОЗИЦИИ (СЛОТЫ) ---
Всего требуется позиций: 60
Заполнено позиций: 60
Осталось вакантных: 0

--- !!! ПРОБЛЕМНЫЕ БРИГАДЫ ---
Всего бригад в плане (week×shift×machine): 24
Укомплектовано (N/N): 24
Неукомплектовано (M/N): 0
Не запущено (0/N): 0


## 7. Проблемные бригады

In [54]:
scheduler_report.problem_brigades().head()

,week,shift,machine_id,machine_type,assigned,required,missing,status
